In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [18]:
# data loading in appropriate formate

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")


11501568/11490434 [==============================] - 0s 0us/step


In [19]:
y_train=tf.keras.utils.to_categorical(y_train)

In [26]:
class CONV(tf.keras.layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CONV, self).__init__()
        self.conv = tf.keras.layers.Conv2D(out_channels, kernel_size, padding="same")

    def call(self, input_tensor):
        x = self.conv(input_tensor)
        x = tf.nn.relu(x)
        return x

In [31]:
class inception(tf.keras.layers.Layer):
    def __init__(self,f1,f3,f5):
        super(inception, self).__init__()
        self.conv1=CONV(f1,1)
        self.conv2=CONV(f3,3)
        self.conv3=CONV(f5,5)
        self.conv4=CONV(f1,1)
        self.max3=tf.keras.layers.MaxPool2D(3,padding='same',strides=1)

    def call(self, input_tensor):
        a=self.conv1(input_tensor)
        b=self.conv2(a)
        c=self.conv3(a)
        d=self.conv4(self.max3(input_tensor))
        x=tf.keras.layers.Concatenate()([a,b,c,d])
        return x

In [32]:
#model subclassing with convolution layer.

class MyModel(tf.keras.Model):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        # define all layers in init
        # Layer of Block 1
        self.c1 = CONV(32,3)
        self.incep1 = inception(10,10,10)

        #fully connected layer
        self.flat = tf.keras.layers.Flatten()
        self.d1 = keras.layers.Dense(20, activation='relu')
        self.d2 = keras.layers.Dense(num_classes)


    def call(self, input_tensor):
        # forward pass: block 1 
        x = self.c1(input_tensor)
        x = self.incep1(x)

        # forward pass: FC layer
        x=self.flat(x)
        x = self.d1(x)
        return self.d2(x)


In [33]:
model=MyModel(10)

"""loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])"""
model.compile(
          loss      = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
          metrics   = tf.keras.metrics.CategoricalAccuracy(),
          optimizer = tf.keras.optimizers.Adam())

In [34]:
model.build(input_shape=(None, 28, 28, 1))
model.call(tf.keras.Input(shape=(28, 28, 1)))
model.summary()
#no of parameter in conv  = ((m*n*d)+1)*k) for ex ((3*3*1)+1)*32 = 320
# ((3*3*32)+1)*64) = 18496

Model: "my_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_8 (CONV)               (None, 28, 28, 32)        320       
                                                                 
 inception_1 (inception)     (None, 28, 28, 40)        3750      
                                                                 
 flatten_3 (Flatten)         (None, 31360)             0         
                                                                 
 dense_6 (Dense)             (None, 20)                627220    
                                                                 
 dense_7 (Dense)             (None, 10)                210       
                                                                 
Total params: 631,500
Trainable params: 631,500
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.fit(x_train, y_train, batch_size=128,epochs=2,validation_split=0.2)

Epoch 1/2
375/375 [==============================] - 185s 491ms/step - loss: 0.2674 - categorical_accuracy: 0.9194 - val_loss: 0.0896 - val_categorical_accuracy: 0.9735
Epoch 2/2
375/375 [==============================] - 179s 478ms/step - loss: 0.0706 - categorical_accuracy: 0.9785 - val_loss: 0.0647 - val_categorical_accuracy: 0.9808
